# Taggable interface

In [1]:
using TenetCore
using Networks
using QuantumTags

`Taggable` is the interface that allows a `Network` to tag its vertices and edges with `Tag`s.

`GenericTensorNetwork` is a concrete type that wraps `SimpleTensorNetwork` and implements `Taggable` (along other interfaces).

In [2]:
tn = GenericTensorNetwork()

GenericTensorNetwork (#tensors=0, #inds=0)

In [3]:
Γa = Tensor([1 0; 0 1], [Index(:i), Index(:j)])
Γb = Tensor([1 0; 0 1], [Index(:j), Index(:k)])
addtensor!(tn, Γa)
addtensor!(tn, Γb)

SimpleTensorNetwork (#tensors=2, #inds=3)

It associates vertices and edges with `Site` and `Link` tags.

`Site` is a `Tag` abstract type that should be thought as a "physical site" in physical lattice. The most basic concrete subtype is `CartesianSite`, which stores a `Tuple` of integers.

In [4]:
@show CartesianSite(1) CartesianSite(2,3)

CartesianSite(1) = (1,)
CartesianSite(2, 3) = (2, 3)


(2, 3)

In other to facilitate writing, there is the `@site_str` macro that translates to a `CartesianSite`.

In [5]:
site"1"

(1,)

In [6]:
site"1,2"

(1, 2)

In [7]:
typeof(site"1")

CartesianSite{1}

In order to tag a vertex with a `Site`, there is the `tag_vertex!` function.

In [8]:
v = vertex_at(tn, Γa)
tag_vertex!(tn, v, site"1")

GenericTensorNetwork (#tensors=2, #inds=3)

Note that since `Tensor`s are associated one-to-one with vertices, they can also be use in substitution of vertices; i.e. you can directly tag `Tensor`s.

In [9]:
tag_vertex!(tn, Γb, site"2")

GenericTensorNetwork (#tensors=2, #inds=3)

`vertex_tags` returns all the tags associated to vertices / `Tensor`s (i.e. all the `Site` tags).

In [10]:
vertex_tags(tn)

2-element Vector{Site}:
 (2,)
 (1,)

In order to retrieve a vertex or tensor with that tag, there are the `vertex_at` and `tensor_at` (aka `tensor(; at)`) functions.

In [11]:
vertex_at(tn, site"1")

Networks.Vertex{Base.UUID}(Base.UUID("2a2f60c2-f6f6-4782-958f-342f53a916e3"))

In [12]:
tensor_at(tn, site"1")

2×2 Tensor{Int64, 2, Matrix{Int64}}:
 1  0
 0  1

In [13]:
tensor(tn; at=site"1") === Γa

true

On the other hand, there are `Link`s: `Tag` abstract type for `Index`s.

The most fundamental `Link` types are `Plug` (used for representing physical input/output open indices) and `Bond` (used for representing virtual bonds connecting different sites).

In [14]:
plug"1"

(1,)

In [15]:
typeof(plug"1")

Plug{CartesianSite{1}}

In [16]:
bond"1-2"

(1,) <=> (2,)

In [17]:
Bond(site"1", site"2")

(1,) <=> (2,)

Just like with `Site`s, `tag!` can be used for tagging `Index` with a `Link`.

In [18]:
e = edge_at(tn, Index(:i))
tag_edge!(tn, e, plug"1")

GenericTensorNetwork (#tensors=2, #inds=3)

In [19]:
tag_edge!(tn, Index(:k), plug"2")
tag_edge!(tn, Index(:j), bond"1-2")

GenericTensorNetwork (#tensors=2, #inds=3)

`edge_tags` retrieves all the `Tag`s associated with edges / `Index`s.

In [20]:
edge_tags(tn)

3-element Vector{Link}:
 (1,)
 (2,)
 (1,) <=> (2,)

Finally, `edge_at` and `ind_at` (aka `ind(; at)`) return the `Edge` and `Index` linked to the tag.

In [21]:
ind_at(tn, plug"1")

Index{Symbol}(:i)

## The `TensorNetwork` underneath

`GenericTensorNetwork` implements both `TensorNetwork` (through the delegation to `SimpleTensorNetwork`) and `Taggable`, but a method from one of the interfaces might mutate the `GenericTensorNetwork` in such a way that the information from the other interface might be invalidated. It is the responsability of the implementor type (i.e. `GenericTensorNetwork`) to fix any inconsistencies.

For example, calling `rmtensor!` on `GenericTensorNetwork` will automatically be delegated to the `SimpleTensorNetwork` and the `Tensor` (and `Vertex`) will be removed. If the `Tensor` is associated to a `Site`, removing the `Tensor` and not the `Site` will create a inconsistency.
Due to that, `GenericTensorNetwork` intercepts the `rmtensor!`, removes the tag and finally calls `rmtensor!` on its delegated field (i.e. on the `SimpleTensorNetwork`).
Another example is that `addvertex!` or `rmvertex!` from the `Network` interface on `SimpleTensorNetwork` would break the one-to-one map between vertices and `Tensor`s, and thus `SimpleTensorNetwork` forbids them.

Summarizing, calling mutating functions of interfaces that are delegated should be safe or forbidden (note for implementors).

In [22]:
γa = similar(Γa)
replace!(tn, Γa => γa)

GenericTensorNetwork (#tensors=2, #inds=3)

In [23]:
tag_at_vertex(tn, vertex_at(tn, Γa))

KeyError: KeyError: key [1 0; 0 1] not found

`site_at(tn, Γa)` fails because `Γa` is no longer in the Tensor Network, but the tag remains associated with the new `Tensor` it has been replaced with.

In [24]:
tag_at_vertex(tn, vertex_at(tn, γa))

(1,)

## Creating a new `Tag` type

Creating a new `Tag` type is extremely easy. You just keep in mind that you might want to overload some methods related to somo other `Tag` if it needs to interact with or act like it.

In [25]:
struct VidalLambda{B} <: QuantumTags.Site
    bond::B
end

QuantumTags.issite(::VidalLambda) = false
QuantumTags.bond(x::VidalLambda) = x.bond
QuantumTags.isbond(x::VidalLambda) = true

In [26]:
Λ = Tensor([1, 1], [Index(:j)])
addtensor!(tn, Λ)
tag_vertex!(tn, Λ, VidalLambda(bond"1-2"))

GenericTensorNetwork (#tensors=3, #inds=3)

In [27]:
tensor(tn; at=VidalLambda(bond"1-2"))

2-element Tensor{Int64, 1, Vector{Int64}}:
 1
 1